# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

In [10]:
import sys

!{sys.executable} -m pip install -q numpy pandas

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [11]:
import numpy as np
import pandas as pd

A_df = pd.read_csv("model-A.tsv", sep="\t").set_index("index")
B_df = pd.read_csv("model-B.tsv", sep="\t").set_index("index")
Q_df = pd.read_csv("cost-Q.tsv", sep="\t").set_index("index")
R_df = pd.read_csv("cost-R.tsv", sep="\t").set_index("index")

A = A_df[["x", "v", "a"]].values.astype(float)
B = B_df[["u"]].values.astype(float)
Q = Q_df[["x", "v", "a"]].values.astype(float)
R = R_df[["u"]].values.astype(float)

M = Q.copy()
tol = 1e-12
max_iters = 200000

for _ in range(max_iters):
    RB = R + B.T.dot(M).dot(B)
    K_tmp = np.linalg.solve(RB, B.T.dot(M).dot(A))
    M_next = Q + A.T.dot(M).dot(A) - A.T.dot(M).dot(B).dot(K_tmp)

    if np.max(np.abs(M_next - M)) < tol:
        M = M_next
        break

    M = M_next

RB = R + B.T.dot(M).dot(B)
K = np.linalg.solve(RB, B.T.dot(M).dot(A))   # shape (1,3)

Submit "control-K-intended.tsv" in Gradescope.

In [12]:
K_df = pd.DataFrame(K, columns=["x", "v", "a"])
K_df.to_csv("control-K-intended.tsv", sep="\t", index=False)

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [13]:
import pandas as pd

log_df = pd.read_csv("qc-log.tsv", sep="\t")

u_check = -(K[0, 0] * log_df["x"] + K[0, 1] * log_df["v"] + K[0, 2] * log_df["a"])

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [14]:
out_df = pd.DataFrame({
    "index": log_df["index"],
    "u_check": u_check
})

out_df.to_csv("qc-check.tsv", sep="\t", index=False)

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [15]:
log_df = pd.read_csv("qc-log.tsv", sep="\t")

X = log_df[["x", "v", "a"]].values.astype(float)     # (n, 3)
u = log_df["u"].values.astype(float).reshape(-1, 1)  # (n, 1)

beta, _, _, _ = np.linalg.lstsq(X, u, rcond=None)     # beta ≈ (-K)^T

K_actual = (-beta.T)                                  # (1, 3)

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [16]:
K_actual_df = pd.DataFrame(K_actual, columns=["x", "v", "a"])
K_actual_df.to_csv("control-K-actual.tsv", sep="\t", index=False)

Submit "control-k-actual.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [17]:
train_df = pd.read_csv("qc-train.tsv", sep="\t")

X = train_df[["x", "v", "a"]].values.astype(float)   # (T, 3)
U = train_df[["u"]].values.astype(float)             # (T, 1)

X_t = X[:-1, :]                                      # (T-1, 3)
X_next = X[1:, :]                                    # (T-1, 3)
U_t = U[:-1, :]                                      # (T-1, 1)

D = np.hstack([X_t, U_t])                            # (T-1, 4)

Theta, _, _, _ = np.linalg.lstsq(D, X_next, rcond=None)  # (4, 3)

A_hat = Theta[:3, :].T                               # (3, 3)
B_hat = Theta[3, :].reshape(3, 1)                    # (3, 1)

Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [18]:
A_out = pd.DataFrame(A_hat, columns=["x", "v", "a"])
A_out.insert(0, "index", ["x", "v", "a"])
A_out.to_csv("model-A-actual.tsv", sep="\t", index=False)

B_out = pd.DataFrame(B_hat, columns=["u"])
B_out.insert(0, "index", ["x", "v", "a"])
B_out.to_csv("model-B-actual.tsv", sep="\t", index=False)

Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

None beyond standard libraries and their documentation (NumPy and Pandas) used for matrix operations and reading/writing TSV files.